# ClickHouse Data Exploration

This notebook is for running ad-hoc, exploratory queries against our ClickHouse trade warehouse.

**Setup (run in your terminal):**
1.  Make sure you are in your project's virtual environment (or install these globally).
2.  Install the required libraries:
    ```bash
    pip install jupyterlab pandas plotly clickhouse-driver
    ```
3.  Run the notebook: `jupyter lab`

In [ ]:
from clickhouse_driver import Client
import pandas as pd
import plotly.graph_objects as go

# --- Connection --- 
CLICKHOUSE_HOST = "localhost"
CLICKHOUSE_PORT = 8123 # This is our clickhouse-01 node
CLICKHOUSE_DB = "default"

print(f"Connecting to ClickHouse at {CLICKHOUSE_HOST}:{CLICKHOUSE_PORT}...")
try:
    client = Client(
        host=CLICKHOUSE_HOST,
        port=CLICKHOUSE_PORT,
        database=CLICKHOUSE_DB
    )
    # Verify connection
    client.execute('SELECT 1')
    print("Connection successful!")
except Exception as e:
    print(f"Connection failed: {e}")

# Helper function to run queries and get a DataFrame
def query_to_df(query: str, params=None):
    print("Running query...")
    try:
        data, columns = client.execute(query, params=params, with_column_types=True)
        col_names = [c[0] for c in columns]
        df = pd.DataFrame(data, columns=col_names)
        print(f"Query successful. Returned {len(df)} rows.")
        return df
    except Exception as e:
        print(f"Query failed: {e}")
        return pd.DataFrame()

## 1. Check Ingestion

Let's check if data is flowing into our tables. We query `ticks_all`, which is the `Distributed` table (our main query entry point).

In [ ]:
query = """
SELECT * FROM default.ticks_all
LIMIT 10
"""
df_head = query_to_df(query)
df_head

In [ ]:
query = """
SELECT 
    symbol, 
    count() AS num_ticks
FROM default.ticks_all
GROUP BY symbol
ORDER BY num_ticks DESC
"""
df_counts = query_to_df(query)
df_counts

## 2. Test Deduplication (ReplacingMergeTree)

Let's check the fast (dirty) count vs. the slow (clean) count using `FINAL`. This is the same benchmark our API will run.

In [ ]:
symbol = 'AAPL'

# 1. Fast, raw count (may include duplicates)
query_raw = "SELECT count() FROM default.ticks_dedup WHERE symbol = %(sym)s"
df_raw = query_to_df(query_raw, params={'sym': symbol})
print(f"Raw count for {symbol}: {df_raw.iloc[0,0]}")

# 2. Slow, accurate count (forces deduplication)
query_final = "SELECT count() FROM default.ticks_dedup FINAL WHERE symbol = %(sym)s"
df_final = query_to_df(query_final, params={'sym': symbol})
print(f"Final count for {symbol}: {df_final.iloc[0,0]}")

## 3. Test Rollup Table (AggregatingMergeTree)

Let's query our fast 1-minute rollup table. We'll grab the pre-calculated states and then use the `...Merge` functions to finalize them. This is the 'fast path' query for our API.

In [ ]:
symbol = 'AAPL'
limit = 10

query_fast = """
SELECT
    minute,
    symbol,
    argMinMerge(open) AS open,
    maxMerge(high) AS high,
    minMerge(low) AS low,
    argMaxMerge(close) AS close,
    sumMerge(volume) AS volume,
    sumMerge(vwap_pv) / sumMerge(volume) AS vwap
FROM 
    default.trades_1m_agg
WHERE 
    symbol = %(sym)s
GROUP BY 
    symbol, minute
ORDER BY 
    minute DESC
LIMIT %(lim)s
"""

df_fast = query_to_df(query_fast, params={'sym': symbol, 'lim': limit})
df_fast